In [ ]:
%load_ext cmdstanjupyter

In [ ]:
%%stan wiener
functions {
  // GPL-2
  // from https://github.com/cran/RWiener/blob/master/src/rwiener.c
  real wiener_rejection_rng(real a_in, real ter, real z_in, real v_in){
    real dt=1e-15, D=0.005, totaltime=0, startpos=0, 
    Aupper, Alower, radius, lambda, F, prob, tt, dir_, l, s1, s2, tnew, t_delta;
    int uu;
    int finish = 0;
    real t=0, r;
    
    real a = a_in / 10;
    real z = z_in / 10;
    real v =  v_in / 10;
    
    Aupper=a-z;
    Alower=-z;
    radius=fmin(abs(Aupper),abs(Alower));

    while (!finish) {
      if (v==0){
        lambda = 0.25*D*pi()*pi()/(radius*radius);
        F=1;
        prob = .5;
      } 
      else {
        lambda = 0.25*v*v/D + 0.25*D*pi()*pi()/(radius*radius);
        F=D*pi()/(radius*v);
        F=F*F/(1+F*F);
        prob=exp(radius*v/D);
        prob=prob/(1+prob);
      }
      r = uniform_rng(0,1);
      dir_= r<prob ? 1 : -1;
      l=-1;
      s2=0;

      while (s2>l) {
        s2 = uniform_rng(0,1);
        s1 = uniform_rng(0,1);
        tnew=0;
        t_delta=0;
        uu=0;

        while ( (abs(t_delta)>dt) || (!uu) ) {
          tt = 2*++uu+1;
          t_delta = tt * (uu%2?-1:1) * pow(s1,(F*tt*tt));
          tnew += t_delta;
        }

        l = 1 + pow(s1,-F) * tnew;
      }/*end while (s2>l) */

      totaltime+=abs(log(s1))/lambda;
      dir_=startpos+dir_*radius;

      if (dir_+dt>Aupper) {
        //*t=totaltime+ter;
        //*x=1;
        t = totaltime+ter;
        finish=1;
        return t;
      }
      else {
        if (dir_-dt<Alower) {
          //*t=totaltime+ter;
          //*x=0;
          t = -(totaltime+ter);
          finish=1;
          return t; 
        }
        else {
          startpos=dir_;
          radius=fmin(abs(Aupper-startpos),abs(Alower-startpos));
        }
      }
    } /*end while (!finish) */
    return t; // to avoid compiler warnings
  }   
  
  
  real wiener_rng(real alpha, real tau, real beta, real delta)
  {
    return wiener_rejection_rng(alpha, tau, beta*alpha, delta);
  }
}
  
generated quantities {
    array[20] real ans;
    for (i in 1:20){
      ans[i] = wiener_rng(1.5, 5.0, 0.3, -1.2);
      
  }
}

In [ ]:
fit = wiener.sample(show_console=True, iter_sampling=10)